In [ ]:
# Step 1: Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

# Step 2: Generate Dataset
np.random.seed(1)
X, Y = make_moons(n_samples=400, noise=0.2)
Y = Y.reshape((Y.shape[0], 1))

# Step 3: Visualize Dataset
plt.scatter(X[:, 0], X[:, 1], c=Y[:, 0], s=40, cmap=plt.cm.Spectral)
plt.title("Planar Dataset")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

# Step 4: Activation Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

# Step 5: Initialize Parameters
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(2)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

# Step 6: Forward Propagation
def forward_propagation(X, parameters):
    W1, b1, W2, b2 = parameters.values()
    Z1 = np.dot(W1, X.T) + b1
    A1 = tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    return A2, {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}

# Step 7: Compute Cost
def compute_cost(A2, Y):
    m = Y.shape[0]
    logprobs = np.multiply(np.log(A2), Y.T) + np.multiply(np.log(1 - A2), 1 - Y.T)
    return -np.sum(logprobs) / m

# Step 8: Backward Propagation
def backward_propagation(parameters, cache, X, Y):
    m = X.shape[0]
    W2 = parameters['W2']
    A1, A2 = cache['A1'], cache['A2']
    dZ2 = A2 - Y.T
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

# Step 9: Update Parameters
def update_parameters(parameters, grads, learning_rate=1.2):
    for key in parameters.keys():
        parameters[key] -= learning_rate * grads['d' + key]
    return parameters

# Step 10: Model Training
def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):
    np.random.seed(3)
    n_x, n_y = X.shape[1], Y.shape[1]
    parameters = initialize_parameters(n_x, n_h, n_y)
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads)
        if print_cost and i % 1000 == 0:
            print(f"Cost after iteration {i}: {cost:.4f}")
    return parameters

# Step 11: Predict Function
def predict(parameters, X):
    A2, _ = forward_propagation(X, parameters)
    return (A2 > 0.5).astype(int).T

# Step 12: Train & Evaluate
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
parameters = nn_model(X_train, Y_train, n_h=4, num_iterations=10000, print_cost=True)
train_preds = predict(parameters, X_train)
test_preds = predict(parameters, X_test)
print(f"Train Accuracy: {accuracy_score(Y_train, train_preds) * 100:.2f}%")
print(f"Test Accuracy: {accuracy_score(Y_test, test_preds) * 100:.2f}%")

# Step 13: Plot Decision Boundary
def plot_decision_boundary(model, X, Y):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model(parameters, np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=Y[:, 0], cmap=plt.cm.Spectral)
    plt.title("Decision Boundary")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.show()

plot_decision_boundary(predict, X, Y)
